In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from datetime import datetime
from train import *
from loss import *
from accuracy import *
from model import *

import cifar_input
import os.path
import time
import numpy as np
import tensorflow as tf

In [ ]:
max_steps = 20000
train_dir = 'cifar10_vgg_model/'
batch_size = 128
log_device_placement = False

In [ ]:
with tf.Graph().as_default():
    global_step = tf.contrib.framework.get_or_create_global_step()
    
    # Generating images and its labels
    # build_inputs('cifar10/cifar100', cifar dataset dir, batch size, mode)
    images, labels = cifar_input.build_input('cifar10', '../../cifar/cifar10/data_batch*', batch_size, 'train')
    
    # Creating graph. NUM_CLASESS=10 (CIFAR-10) or NUM_CLASESS=100 (CIFAR-100)
    logits = inference(images, NUM_CLASSES=10)
    
    # Loss/Error and Accuracy
    losses = loss(logits, labels)
    accuracies = accuracy(logits, labels)
    
    # Our train_op (Only minimizing loss)
    train_op = train(losses, global_step, batch_size)
    
    
    # SessionRunHook. Logging will be done each x steps.
    class _LoggerHook(tf.train.SessionRunHook):
        
      def begin(self):
        self._step = -1
        # Creating train_dir if it does not exist and writing to log file
        if not os.path.exists(train_dir):
            os.makedirs(train_dir)
        open(train_dir+'training_data.csv', 'w').close()
        f = open(train_dir+"log.txt",'ab')
        f.write('\n\n==== Run ===\nInfo: VGG\n')
        f.close()

      def before_run(self, run_context):
        # Increment step, reset start_time, and asking for loss and accuracy tensor
        self._step += 1
        self._start_time = time.time()
        return tf.train.SessionRunArgs([losses, accuracies])

      def after_run(self, run_context, run_values):
        duration = time.time() - self._start_time # Calculating time
        loss_value = run_values.results[0]
        accuracy_value = run_values.results[1]
        # Printing log, accuracy, and loss
        if self._step % 10 == 0:
          print("{0}: step {1}, error = {2:.4f}, accuracy = {3:.4f}. ({4:.3f} sec/step)\n".format(
              datetime.now(), self._step, loss_value, accuracy_value, float(duration)))
                
          f = open(train_dir+"log.txt",'ab')
          f.write("{0}: step {1}, error = {2:.4f}, accuracy = {3:.4f}. ({4:.3f} sec/step)\n".format(
              datetime.now(), self._step, loss_value, accuracy_value, float(duration)))
          f.close()
                
          f = open(train_dir+"training_data.csv",'ab')
          f.write('{0},{1},{2}\n'.format(self._step, loss_value, accuracy_value))
          f.close()
            
    with tf.train.MonitoredTrainingSession(checkpoint_dir=train_dir,
                                           hooks=[tf.train.StopAtStepHook(last_step=max_steps),
                                                  tf.train.NanTensorHook(losses),
                                                  _LoggerHook()],save_checkpoint_secs=30, 
                                           config=tf.ConfigProto(
                                               log_device_placement=log_device_placement)) as mon_sess:
        while not mon_sess.should_stop():
            mon_sess.run(train_op)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into cifar10_vgg_model/model.ckpt.
2017-03-14 02:53:13.846137: step 0, error = 15.2894, accuracy = 0.0859. (2.659 sec/step)

2017-03-14 02:53:16.220850: step 10, error = 15.3675, accuracy = 0.0469. (0.306 sec/step)

2017-03-14 02:53:19.257188: step 20, error = 15.0903, accuracy = 0.1250. (0.305 sec/step)

2017-03-14 02:53:22.428776: step 30, error = 14.9678, accuracy = 0.1094. (0.333 sec/step)

2017-03-14 02:53:26.032656: step 40, error = 14.8845, accuracy = 0.0938. (0.333 sec/step)

2017-03-14 02:53:29.620623: step 50, error = 14.7651, accuracy = 0.0938. (0.385 sec/step)

2017-03-14 02:53:33.244579: step 60, error = 14.6681, accuracy = 0.1328. (0.364 sec/step)

2017-03-14 02:53:36.773400: step 70, error = 14.5879, accuracy = 0.0703. (0.367 sec/step)

2017-03-14 02:53:40.410093: step 80, error = 14.5021, accuracy = 0.0859. (0.389 sec/step)

INFO:tensorflow:Saving checkpoints for 90 into cifar10_vgg_mod